In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
import re
import sys

import os
print(os.getcwd())   # Prints the current working directory

In [ ]:
# Provide the new path here

os.chdir('C:\\Users\\user\\Desktop\\Twitter NLP Classification Model\\DATASET\\dataset 31-05-2020')
df = pd.read_csv('result_prepro3.csv')

In [ ]:
####
# RANDOM SAMPLE
###
chosen_idx = np.random.choice(len(df), replace=False, size=10000)
df_sample = df.iloc[chosen_idx]

In [ ]:
df_unigram = df.copy() ## USING ORIGINAL DATA 
# df_unigram = df_sample.copy() ## USING SAMPLE DATA
df_unigram = df_unigram.assign(Text6 = df['Text5'])
df_unigram = df_unigram.assign(Binary1 = 0 )
df_unigram = df_unigram.assign(Binary2 = 0 )
df_unigram['Binary1'] = df_unigram['Class'].apply(lambda x: 0 if 'hoax' in x else 1) #non-hoax
df_unigram['Binary2'] = df_unigram['Class'].apply(lambda x: 1 if 'hoax' in x else 0) #hoax
df_unigram = df_unigram[['Text6','Binary1','Binary2']]

df_unigram.dropna(subset=['Text6'], inplace = True) # exclude NA values in Text6

df_bigram = df_unigram.copy()
df_trigram = df_unigram.copy()

In [ ]:
print(df_unigram.head())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_unigram = TfidfVectorizer(ngram_range=(1,1)) 
tfidf_bigram = TfidfVectorizer(ngram_range=(2,2))
tfidf_trigram = TfidfVectorizer(ngram_range=(3,3))

In [ ]:
vectors_unigram = tfidf_unigram.fit_transform(df_unigram['Text6'].values.astype('U'))
vectors_bigram = tfidf_bigram.fit_transform(df_bigram['Text6'].values.astype('U'))
vectors_trigram = tfidf_trigram.fit_transform(df_trigram['Text6'].values.astype('U'))

In [ ]:
# print(vectors_unigram.size)

In [ ]:
print(vectors_unigram.shape)
print(vectors_bigram.shape)
print(vectors_trigram.shape)

In [ ]:
print(vectors_unigram.shape[1])

In [ ]:
############################################################################
# IMPORTANT , PLEASE SET OUTPUT to be either 1 or 2 columns deliberately
############################################################################

##Output combine binary 1 + binary 2 [SELECT 1 COLUMN of OUTPUT]
# output = df_unigram[['Binary1','Binary2']].to_numpy()

## Output is Binary2 [SELECT 2 COLUMNs of OUTPUT]
output = df_unigram[['Binary2']].to_numpy()

In [ ]:
print(df_unigram.shape)
print(df_bigram.shape)
print(df_trigram.shape)

In [ ]:
# print(output.size)

In [ ]:
print(output.shape)

In [ ]:
# print(output[0].shape)

In [ ]:
print(vectors_unigram[:,0:10].shape) #[row,columns] # select all rows, while selecting columns with itterate 0-10 [10 columns]
print(output[:,0:2]) #[row,columns] # select all rows, while selecting columns with itterate 0-2 [2 columns]

In [ ]:
feature_names_unigram = tfidf_unigram.get_feature_names()
print(len(feature_names_unigram))

feature_names_bigram = tfidf_bigram.get_feature_names()
print(len(feature_names_bigram))

feature_names_trigram = tfidf_trigram.get_feature_names()
print(len(feature_names_trigram))

In [ ]:
########################################
# Displaying N-gram word results (Cant display all due to memory issue, select one & uncomment it)
#######################################
# print(feature_names_unigram)
# print(feature_names_bigram)   
# print(feature_names_trigram)

In [ ]:
vectors_unigram=vectors_unigram # selecting all unigram tf-idf score (24446 scores)
print(vectors_unigram.shape)

vectors_bigram=vectors_bigram # selecting all bigram tf-idf scores (118397 scores)
print(vectors_bigram.shape) 

vectors_trigram=vectors_trigram  # selecting all trigram tf-idf score (114664 scores)
print(vectors_trigram.shape)

In [ ]:
#Variable Input , output and number of neurons for ANN

row_train_end = round(vectors_unigram.shape[0] * 0.8)
row_test_end = vectors_unigram.shape[0]

# Unigram
input_unigram_training = vectors_unigram[0:row_train_end]                #80%
output_unigram_training = output[0:row_train_end]                        #80%
input_unigram_testing = vectors_unigram [row_train_end:row_test_end]     #20%
output_unigram_testing = output[row_train_end:row_test_end]              #20%
print(input_unigram_training.shape)
print(output_unigram_training.shape)
print(input_unigram_testing.shape)
print(output_unigram_testing.shape)

# Bigram
input_bigram_training = vectors_bigram[0:row_train_end]                 #80%
output_bigram_training = output[0:row_train_end]                        #80%
input_bigram_testing = vectors_bigram [row_train_end:row_test_end]      #20%
output_bigram_testing = output[row_train_end:row_test_end]              #20%

# Trigram
input_trigram_training = vectors_trigram[0:row_train_end]                #80%
output_trigram_training = output[0:row_train_end]                        #80%
input_trigram_testing = vectors_trigram [row_train_end:row_test_end]     #20%
output_trigram_testing = output[row_train_end:row_test_end]              #20%


uni_neurons = vectors_unigram.shape[1]  # input size neuron for unigram
bi_neurons = vectors_bigram.shape[1]    # input size neuron for bigram
tri_neurons = vectors_trigram.shape[1]  # input size neuron for trigram

In [ ]:
print(uni_neurons) # input size for unigram
print(bi_neurons) # input size for bigram
print(tri_neurons) # input size for trigram

In [ ]:
###############################################################################
# SET UNI-GRAM, BI-GRAM, TRI-GRAM IN THIS PART
###############################################################################

n_grams = 1 # UNI-GRAM
# n_grams = 2 # BI-GRAM
# n_grams = 3 # TRI-GRAM

if n_grams == 1:
    input_training_ANN = input_unigram_training 
    input_testing_ANN = input_unigram_testing
    output_training_ANN = output_unigram_training
    output_testing_ANN = output_unigram_testing
    input_neurons = uni_neurons
    print("Unigram Selected ")
    
elif n_grams == 2:
    input_training_ANN = input_bigram_training
    input_testing_ANN = input_bigram_testing
    output_training_ANN = output_bigram_training
    output_testing_ANN = output_bigram_testing
    input_neurons = bi_neurons
    print("Bigram Selected ")
    
elif n_grams == 3:
    input_training_ANN = input_trigram_training
    input_testing_ANN = input_trigram_testing
    output_training_ANN = output_trigram_training
    output_testing_ANN = output_trigram_testing
    input_neurons = tri_neurons
    print("Trigram Selected ")

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import seaborn as sns


import tensorflow as tf 
#import tensorflow_text as text 

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

from tensorflow.keras import initializers
from tensorflow.python.ops.init_ops import glorot_normal_initializer
from tensorflow.python.ops.init_ops import glorot_uniform_initializer
from tensorflow.python.ops.init_ops import he_normal
from tensorflow.python.ops.init_ops import he_uniform
from tensorflow.python.ops.init_ops import truncated_normal_initializer

import numpy as np
import matplotlib.pyplot as plt
import array 
import pandas as pd

from scipy.sparse import coo_matrix



In [ ]:
# tf.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=123)
# BEGIN CODE TENSORFLOW 
seed_id_number = 123
tf.random.set_seed(seed_id_number)
def build_model():
    model = keras.Sequential([    
    layers.Dense(500, activation='relu' ,
                 input_shape=[input_neurons],
#                  kernel_initializer = truncated_normal_initializer(mean=0.0, stddev=0.05, seed=seed_id_number),
                 kernel_initializer = he_normal(seed=seed_id_number),
                 bias_initializer = he_normal(seed=seed_id_number)
                ), #1st hidden layer
    layers.Dense(350, activation='relu',
                 kernel_initializer = he_normal(seed=seed_id_number),
                 bias_initializer = he_normal(seed=seed_id_number)
                ), #2nd hidden layer
    layers.Dense(200, activation='relu',
                 kernel_initializer = he_normal(seed=seed_id_number),
                 bias_initializer = he_normal(seed=seed_id_number)
                ), #3rd hidden layer
    layers.Dense(1, activation ='sigmoid',
                kernel_initializer = glorot_normal_initializer(seed=seed_id_number),
                bias_initializer = glorot_normal_initializer(seed=seed_id_number)
                ), #output layer
    layers.Dropout(0.5)
    ])

  # (Learning Rate, Momentum,nesterov)
    optimizer = tf.keras.optimizers.SGD(0.01,0.9,False) 

    model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error','accuracy'])
    return model 

In [ ]:
#Create ANN Model
model = build_model()
model.summary()

In [ ]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback): #Early Stopping
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')
##Set training Iterations
# EPOCHS = 10
EPOCHS = 200
#Run Training Model in each unigram, bigram, trigram
history = model.fit(
    input_training_ANN, output_training_ANN, #80%
    epochs=EPOCHS, 
    verbose=0, #0 or 1. Verbosity mode. 0 = silent, 1 = progress bar.
    validation_split = 0.2, validation_steps=20,
    callbacks=[PrintDot()])

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail() 

In [ ]:
#Print Weight and Bias for each layers
weight_1, bias_1 = model.layers[0].get_weights()
weight_2, bias_2 = model.layers[1].get_weights()
weight_3, bias_3 = model.layers[2].get_weights()
weight_4, bias_4 = model.layers[3].get_weights()

print('1st layer weight :\n',weight_1)
print('1st layer bias : \n',bias_1)

print('2nd layer weight : \n',weight_2)
print('2nd layer bias : \n',bias_2)

print('3rd layer weight : \n',weight_3)
print('3rd layer bias : \n',bias_3)

print('4rd layer weight : \n',weight_4)
print('4rd layer bias : \n',bias_4)

In [ ]:
#Set Plot 'Module'
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    
    ## Plot Target Output Graph
    fig = plt.figure()
    fig.suptitle(' [TARGET OUTPUT]')
    plt.plot(output_testing_ANN)
    
    ##Plot Mean Absolute Error vs Epoch Graph (train & validation)
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [MPG]')
    plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Validation Error')
    plt.legend()
    
    ##Plot Mean Square Error vs Epoch graph (train & validation)
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$MPG^2$]')
    plt.plot(hist['epoch'], hist['loss'],
           label='Train Error')
    plt.plot(hist['epoch'], hist['val_loss'],
           label = 'Validation Error')
    plt.legend()
    
    ##Plot Validation (Train & validation)
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.plot(hist['epoch'], hist['accuracy'],
            label='Train Accuracy')
    plt.plot(hist['epoch'], hist['val_accuracy'],
            label='Validation Accuracy')
    plt.legend()

##Calling plot 'Module'   
plot_history(history)

##########################
#Run Testing Model 
##########################
test_predictions = model.predict(input_testing_ANN)

print(len(test_predictions))
test_predictions1 = test_predictions[1]
print(test_predictions1)

##Plot Prediction Output Graph
test_predictions[test_predictions >= 0.5] = 1
test_predictions[test_predictions < 0.5] = 0

##############################
#Print Output Unigram Testing
##############################
print(output_testing_ANN)    #80
print(test_predictions)

plt.figure()
plt.suptitle('Output Prediction')
plt.plot(test_predictions)

print(('Test Predictions = {}').format(test_predictions))

test_predictions[test_predictions >= 0.5] = 1
test_predictions[test_predictions < 0.5] = 0
print(test_predictions)

##Plot Unigram Prediction Error [MPG] graph 
error = test_predictions - output_testing_ANN
plt.figure()
plt.hist(error, bins = 25, label = "Binary1")
plt.xlabel("Prediction Error [MPG]")
plt.ylabel("Count")
plt.legend(loc="upper right")
plt.show()

In [ ]:
print(type(test_predictions))
print(type(input_testing_ANN)) 
print(type(df[40516:50645]))
# df_testing = df_unigram.iloc[40516:50645,:].copy()
df_testing = df_unigram.iloc[row_train_end:row_test_end,:].copy()


In [ ]:
df_testing2 =  pd.DataFrame(test_predictions[:,0] , columns =['Test Predictions Binary1'])
############################################################################
## USE THIS FOR OUTPUT with 2 COLUMNS
# df_testing2['Test Predictions Binary2'] =  pd.DataFrame(test_predictions[:,1])
############################################################################
df_testing2['Test Predictions Binary2'] =  pd.DataFrame(test_predictions[:,0])
df_testing2['Text5'] = df_testing['Text6'].to_numpy()
print(df_testing2.head())

In [ ]:
df_testing3 = df_testing2[df_testing2['Test Predictions Binary1']==0] # Filter Non-Hoax Predictions
# df_testing3 = df_testing2[df_testing2['Test Predictions Binary2']==1] # Filter Hoax Prediction
print(df_testing3.head())
print(df_testing3.size)

In [ ]:
words = list(df_testing3['Text5'])
words = np.array(words)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()   
cv_fit=cv.fit_transform(words) 
word_list = cv.get_feature_names();    
count_list = cv_fit.toarray().sum(axis=0)    

words_hoax = dict(zip(word_list,count_list))
# print(words_hoax)

In [ ]:
# df_words_hoax = pd.DataFrame([words_hoax])
df_words_hoax = pd.DataFrame(words_hoax.items(), columns=['Hoax Words', 'Frequency'])
print(df_words_hoax.tail())

In [ ]:
df_words_hoax.to_csv('words_hoax.csv')

In [ ]:
df_testing['Test Predictions Binary2'] = df_testing2['Test Predictions Binary2'].to_numpy()

In [ ]:
# print(df_testing.head())

In [ ]:
df_testing.to_csv('result_prediction.csv')

In [ ]:
#confusion Matrix

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
matrix = confusion_matrix(test_predictions.argmax(axis=1),output_testing_ANN.argmax(axis=1))
print(matrix)

In [ ]:
if n_grams == 1:
    vectors_testing_ANN = vectors_unigram[row_train_end:row_test_end]
    tfidf_ANN = tfidf_unigram
elif n_grams == 2:
    vectors_testing_ANN = vectors_bigram[row_train_end:row_test_end]
    tfidf_ANN = tfidf_bigram
elif n_grams == 3:
    vectors_testing_ANN = vectors_trigram[row_train_end:row_test_end]
    tfidf_ANN = tfidf_trigram

In [ ]:
from sklearn.feature_selection import chi2

In [ ]:
chi2score = chi2(vectors_testing_ANN, test_predictions)[0] # 1 dimension output
# chi2score = chi2(vectors_testing_ANN, test_predictions[:,1])[0] # 2 dimension output

In [ ]:
# print(tfidf_ANN.get_feature_names())

In [ ]:
plt.figure(figsize=(15,10))
wscores = list(zip(tfidf_ANN.get_feature_names(), chi2score)) #list
wscores1 = [row for row in wscores if not np.isnan(row[1])]  # remove NAN value
wchi2 = sorted(wscores, key=lambda x:x[1]) #with NAN values
# wchi2 = sorted(wscores1, key=lambda x:x[1]) #without NAN values
topchi2 = list(zip(*wchi2[-30:]))
print(topchi2)
x = range(len(topchi2[1]))
labels = topchi2[0]
plt.barh(x,topchi2[1], align='center', alpha=0.2)
plt.plot(topchi2[1], x, '-o', markersize=5, alpha=0.8)
plt.yticks(x, labels)
plt.xlabel('$\chi^2$')

In [ ]:
from sklearn.metrics import confusion_matrix
def plot_cm(labels, predictions, p = 0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))

In [ ]:
print(test_predictions)
plot_cm(output_testing_ANN[:,0], test_predictions[:,0])